## RNG Analysis using LSTM

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import random
import os

In [ ]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
def patterned_rng(n=50000):
    """Artificial weak RNG with a simple pattern."""
    seq = []
    x = 0
    for _ in range(n):
        # next bit depends deterministically on previous two
        x = (x + 1) % 4
        seq.append(1 if x in (1,2) else 0)
    return np.array(seq, dtype=np.float32)


def rng_builtin_random(n=50000):
    """Python's built-in RNG (LCG-based, weak)"""
    seq = [random.getrandbits(1) for _ in range(n)]
    return np.array(seq, dtype=np.float32)

def rng_mersenne_twister(n=50000):
    """NumPy RNG (MT19937 - medium strength)"""
    seq = np.random.randint(0, 2, size=n)
    return np.array(seq, dtype=np.float32)

def rng_secure(n=50000):
    """Cryptographically Secure RNG (os.urandom)"""
    bits = []
    for _ in range(n // 8):
        byte = os.urandom(1)
        bits.extend([(byte[0] >> i) & 1 for i in range(8)])
    return np.array(bits[:n], dtype=np.float32)

In [ ]:

def create_dataset(seq, window_size=16):
    X, y = [], []
    for i in range(len(seq) - window_size):
        X.append(seq[i:i+window_size])
        y.append(seq[i+window_size])
    X = torch.tensor(X).unsqueeze(-1)  # (samples, seq_len, 1)
    y = torch.tensor(y).unsqueeze(-1)  # (samples, 1)
    return X, y

In [ ]:
class LSTM_RNG(nn.Module):
    def __init__(self, hidden_size=64):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return self.sigmoid(out)

def train_lstm(X_train, y_train, epochs=10):
    model = LSTM_RNG()
    dataset = TensorDataset(X_train, y_train)
    loader = DataLoader(dataset, batch_size=64, shuffle=True)
    opt = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.BCELoss()

    for epoch in range(epochs):
        for xb, yb in loader:
            opt.zero_grad()
            preds = model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            opt.step()
        print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item():.4f}")
    return model

def evaluate(model, X_test, y_test):
    with torch.no_grad():
        preds = model(X_test)
        acc = ((preds > 0.5).float() == y_test).float().mean().item()
    return acc

In [ ]:
# Display sample outputs for each RNG
rng_functions = {
    "Built-in RNG (LCG)": rng_builtin_random,
    "Mersenne Twister": rng_mersenne_twister,
    "Secure RNG (os.urandom)": rng_secure,
    "Patterned RNG": patterned_rng
}

print("\n===== Sample RNG Outputs (first 64 bits) =====")
for name, func in rng_functions.items():
    seq = func(64)  # generate just 64 bits
    print(f"\n{name}:")
    print(seq.astype(int))  # convert to int for readability

In [ ]:
# -------- Experiment Runner --------
def run_experiment(rng_func, name, window=64):
    seq = rng_func()
    X, y = create_dataset(seq, window)
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    print(f"\nTraining model for {name}...")
    model = train_lstm(X_train, y_train)
    acc = evaluate(model, X_test, y_test)
    print(f"{name} Prediction Accuracy: {acc*100:.2f}%")
    return acc

# -------- Run All RNGs --------
results = {}
rng_list = [
    ("Patterned RNG", patterned_rng),
    ("Built-in RNG (LCG)", rng_builtin_random),
    ("Mersenne Twister", rng_mersenne_twister),
    ("Secure RNG (os.urandom)", rng_secure)
]

for name, func in rng_list:
    results[name] = run_experiment(func, name)

# -------- Visualization --------
plt.figure(figsize=(7,4))
plt.bar(results.keys(), [v*100 for v in results.values()], color=['red','orange','green'])
plt.title("AI Predictability on Different RNGs")
plt.ylabel("Prediction Accuracy (%)")
plt.ylim(45,70)
for i, v in enumerate(results.values()):
    plt.text(i, v*100 + 1, f"{v*100:.1f}%", ha='center')
plt.show()